In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




In [4]:
df=pd.read_csv('spotify_millsongdata.csv')

In [5]:
df=df.sample(20000).drop('link', axis=1).reset_index(drop=True)


In [6]:
df.groupby('artist').count()

,song,text
artist,,
'n Sync,34,34
ABBA,37,37
Ace Of Base,22,22
Adam Sandler,24,24
Adele,18,18
...,...,...
Zoegirl,11,11
Zornik,4,4
Zox,8,8


NOW that we have assigned the link to another column we can drop them


In [7]:
df.shape

(20000, 3)

In [29]:
artist_list=df.groupby('artist').count()

Now we are having 643 artist list in our dataset


In [30]:
artist_list.count()

song    634
text    634
dtype: int64

In [33]:
df['song']=df['song'].str.lower()

In [34]:

p=df['text'].str.count('\n').sum()

In [35]:
df['text'] = df['text'].apply(lambda x: str(x).lower().replace(r'^\w\s',' ').replace(r'\n',' ',))


In [36]:
df.columns

Index(['artist', 'song', 'text'], dtype='object')

In [37]:

# import nltk
# it is a nlp library , it is used to process the text data
# from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

In this the similar words which are having same meaning but differ in spelling are changed into one common word


In [38]:
def token(text):
    token=nltk.word_tokenize(text)
    a=[stemmer.stem(word) for word in token]
    return " ".join(a)
    

In [39]:
# # import nltk
# # nltk.download('punkt')
# token(df['text'][0])
# token('hello world beautiful beauty')

In [40]:
df['text'].apply(lambda x:token(x))

0        summer 's gone and love ha wither i believ in ...
1        i do n't know if i can yell ani louder how man...
2        written by john curulewski , paul beaver , b. ...
3        februari morn , my car wo n't start today i tu...
4        [ q-tip ] '' yo . nigga be on the mic , they b...
                               ...                        
19995    took a boat sunday , down by the sea it just f...
19996    na na na na oh oh na na na na oh oh na na na n...
19997    wait in line 'till your time tick clock everyo...
19998    there 's a fragil game you play if the ghost o...
19999    ring ! ring ! it 's 7:00 a.m. ! move why'self ...
Name: text, Length: 20000, dtype: object

In [41]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity



tdif=TfidfVectorizer(analyzer='word', stop_words='english')

In [42]:
matrix=tdif.fit_transform(df['text'])

In [43]:
# from sklearn.metrics.pairwise import cosine_similarity
# from scipy.sparse import csr_matrix

# # Assuming 'matrix' is a sparse matrix in CSR format
# sparse_matrix = csr_matrix(matrix)

# # Compute cosine similarity using sparse matrices
similar = cosine_similarity(matrix)

In [44]:
similar[0]

array([1.00000000e+00, 1.87212859e-02, 0.00000000e+00, ...,
       4.09751070e-02, 3.32771738e-02, 9.01362266e-04])

In [45]:
df['song']

0                      slither
1               don't leave me
2                     krakatoa
3                   automobile
4                       all in
                 ...          
19995               downstream
19996                the judge
19997      in the waiting line
19998          mark the graves
19999    the magnificent seven
Name: song, Length: 20000, dtype: object

In [47]:
df[df['song']=='all in'].index[0]

4

Now we are making recommder function for getting the songs


In [50]:
def recommder(song_name):
    idx=df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])), key=lambda x: x[1], reverse=True)
    song=[]
    for s_id in distance[1:20]:
        song.append(df.iloc[s_id[0]].song)
    return song
    
    

In [61]:
recommder('')

IndexError: index 0 is out of bounds for axis 0 with size 0